#Итоговый проект

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from os import listdir
import codecs
import pickle                                                                   
from bs4 import BeautifulSoup as BS
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from boilerpipe.extract import Extractor
from sklearn.cluster import DBSCAN
from sklearn.ensemble import IsolationForest
from scipy.sparse import hstack
from scipy.sparse import vstack
from scipy.sparse import hstack
from scipy.sparse import vstack
from scipy.sparse import csr_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier 
from sklearn.ensemble import RandomForestRegressor 
from sklearn.ensemble import GradientBoostingClassifier
from gensim.models import Word2Vec


from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import KFold
import re
%matplotlib inline

In [ ]:
submit = False 

In [ ]:
def save_obj(obj, name ):                                                       
    with open(name + '.pkl', 'wb') as f:                                        
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)                            
                                                                                
def load_obj(name ):                                                            
    with open(name + '.pkl', 'rb') as f:                                        
        return pickle.load(f)       

In [ ]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

def lemmatize(text):
    words = text.split() # разбиваем текст на слова
    res = list()
    for word in words:
        p = morph.parse(word)[0]
        res.append(p.normal_form)

    return ' '.join(res)

In [ ]:
lemmatize('русская аба')

In [5]:
name = 'NumWordsRulesExtractor'
#name = 'ArticleExtractor'
name = 'train_data'+name
train_data = load_obj(name)#Загружаем готовый распарсенный текст уже

In [ ]:
#это парсинг теста и доп фич (meta)
if submit:
    test_data= pd.read_csv('test_groups.csv',dtype=np.int16)

    parser_type = 'NumWordsRulesExtractor'
    def save_obj(obj, name ):                                                       
        with open(name + '.pkl', 'wb') as f:                                        
            pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)                            

    def load_obj(name ):                                                            
        with open(name + '.pkl', 'rb') as f:                                        
            return pickle.load(f)                                                   
        
    description = {}                                                                      
    keywords= {}                                                                      
    p = {} 
    h1 = {} 
    h2 = {} 
    h3 = {} 
    h4 = {} 
    h5 = {} 
    h6 = {} 
    test_data['description'] = ''
    test_data['keywords'] = ''

    path = 'content/'                                                               
    texts = {}                                                                      
    print('Start: ' + parser_type)                                                  
    test_data = pd.read_csv('test_groups.csv',dtype=np.int16)                      
    #for filename in ['6770.dat']:                                                  
    for filename in tqdm(listdir(path)):                                            
        doc_id = int(filename.strip('.dat'))                                        
        if doc_id not in test_data.doc_id.values:                                  
            continue                                                                
        try:                                                                        
            with codecs.open(path + filename, 'r', encoding='utf-8', errors='ignore') as f:
                url = f.readline().strip()                                          
                html = f.read()                                                     
                bs = BS(html, 'html.parser')#ищем след страницу                         
                meta = bs.find("meta", {"name":"description"})
                if meta:
                    description[doc_id] = meta.get('content','')
                meta = bs.find("meta", {"name":"keywords"})
                if meta:
                    keywords[doc_id] = meta.get('content','')
                    
                p[doc_id] = 0 
                h1[doc_id] = 0 
                h2[doc_id] = 0 
                h3[doc_id] = 0 
                h4[doc_id] = 0 
                h5[doc_id] = 0 
                h6[doc_id] = 0 

                p[doc_id] += len(bs.find_all('p'))
                h1[doc_id] += len(bs.find_all('h1'))
                h2[doc_id] += len(bs.find_all('h2'))
                h3[doc_id] += len(bs.find_all('h3'))
                h4[doc_id] += len(bs.find_all('h4'))
                h5[doc_id] += len(bs.find_all('h5'))
                h6[doc_id] += len(bs.find_all('h6'))
                
                extractor = Extractor(extractor=parser_type, html=html)             
                s = extractor.getText()                                             
                s=s.replace('\n'," ")                                               
                s=s.replace('\t'," ")                                               
                s=s.replace('\r'," ")                                               
                texts[doc_id] = s                                                   
        except UnicodeDecodeError: 
            print('error {}'.format(filename))                                      
            with codecs.open(path + filename, 'r', 'utf-8') as f:                   
                url = f.readline().strip()                                          
                html = f.read()                                                     
            bs = BS(html, 'html.parser')#ищем след страницу                         
            for s in bs.select('script'):                                           
                s.extract()                                                         
            s = bs.get_text()                                                       
            s=s.replace('\n',"")                                                    
            s=s.replace('\t',"")                                                    
            s=s.replace('\r',"")                                                    
            texts[doc_id] = s                                                       
             
    test_data['p'] = test_data.apply(lambda row: p.get(row.doc_id,0), axis = 1)  
    test_data['h1'] = test_data.apply(lambda row: h1.get(row.doc_id,0), axis = 1)  
    test_data['h2'] = test_data.apply(lambda row: h2.get(row.doc_id,0), axis = 1)  
    test_data['h3'] = test_data.apply(lambda row: h3.get(row.doc_id,0), axis = 1)  
    test_data['h4'] = test_data.apply(lambda row: h4.get(row.doc_id,0), axis = 1)  
    test_data['h5'] = test_data.apply(lambda row: h5.get(row.doc_id,0), axis = 1)  
    test_data['h6'] = test_data.apply(lambda row: h6.get(row.doc_id,0), axis = 1)  
    test_data['text'] = test_data.apply(lambda row: texts[row.doc_id], axis = 1)  
    test_data['description'] = test_data.apply(lambda row: description.get(row.doc_id,''), axis = 1)  
    test_data['keywords'] = test_data.apply(lambda row: keywords.get(row.doc_id,''), axis = 1)  
else:
#     path = 'content/'                                                               
#     description = {}                                                                      
#     keywords= {}                                                                      
#     train_data['description'] = ''
#     train_data['keywords'] = ''
#     #for filename in ['1.dat']:                                                  
#     for filename in tqdm(listdir(path)):                                            
#         doc_id = int(filename.strip('.dat'))                                        
#         if doc_id not in train_data.doc_id.values:                                  
#             continue                                                                
#         with codecs.open(path + filename, 'r', encoding='utf-8', errors='ignore') as f:
#             url = f.readline().strip()                                          
#     #        urls[doc_id] = url                                                       
#             html = f.read()                                                     
#             bs = BS(html, 'html.parser')#ищем след страницу                         
#             meta = bs.find("meta", {"name":"description"})
#             if meta:
#                 description[doc_id] = lemmatize(meta.get('content',''))
#             meta = bs.find("meta", {"name":"keywords"})
#             if meta:
#                 keywords[doc_id] = lemmatize(meta.get('content',''))
#     train_data['description'] = train_data.apply(lambda row: description.get(row.doc_id,''), axis = 1)  
#     train_data['keywords'] = train_data.apply(lambda row: keywords.get(row.doc_id,''), axis = 1)  
    path = 'content/'                                                               
    p = {} 
    h1 = {} 
    h2 = {} 
    h3 = {} 
    h4 = {} 
    h5 = {} 
    h6 = {} 
    #urls = {}                                                                      
#     hrefs_n = {}                                                                      
#     hrefs = {}                                                                      
#     description = {}                                                                      
#     keywords= {}                                                                      
#     train_data['description'] = ''
#     train_data['keywords'] = ''
#     train_data['url'] = ''
#     train_data['hrefs'] = ''
#     train_data['hrefs_n'] = 0
#    for filename in ['15731.dat']:                                                  
    for filename in tqdm(listdir(path)):                                            
        doc_id = int(filename.strip('.dat'))                                        
        if doc_id not in train_data.doc_id.values:                                  
            continue                                                                
        with codecs.open(path + filename, 'r', encoding='utf-8', errors='ignore') as f:
            url = f.readline().strip()                                          
            #urls[doc_id] = re.split(';|,|\.|\_|\-|\*|\/|\&|\?|\=',url)
            html = f.read()                                                     
            bs = BS(html, 'html.parser')#ищем след страницу                         
#             meta = bs.find("meta", {"name":"description"})
#             if meta:
#                 description[doc_id] = lemmatize(meta.get('content',''))
#             meta = bs.find("meta", {"name":"keywords"})
#             if meta:
#                 keywords[doc_id] = lemmatize(meta.get('content',''))
#             hrefs_n[doc_id] = 0 
#             hrefs[doc_id] = []                                                       
           
#             hrefs_bs = bs.find_all(href=True)
#             for href in hrefs_bs:
#                 if href:
#                     if href['href'][0:4] == 'http':
#                         hrefs[doc_id].append(re.split(';|,|\.|\_|\-|\*|\/|\&|\?|\=',href['href']))
#                         hrefs_n[doc_id] +=1
            p[doc_id] = 0 
            h1[doc_id] = 0 
            h2[doc_id] = 0 
            h3[doc_id] = 0 
            h4[doc_id] = 0 
            h5[doc_id] = 0 
            h6[doc_id] = 0 
            
            p[doc_id] += len(bs.find_all('p'))
            h1[doc_id] += len(bs.find_all('h1'))
            h2[doc_id] += len(bs.find_all('h2'))
            h3[doc_id] += len(bs.find_all('h3'))
            h4[doc_id] += len(bs.find_all('h4'))
            h5[doc_id] += len(bs.find_all('h5'))
            h6[doc_id] += len(bs.find_all('h6'))
             
    train_data['p'] = train_data.apply(lambda row: p.get(row.doc_id,0), axis = 1)  
    train_data['h1'] = train_data.apply(lambda row: h1.get(row.doc_id,0), axis = 1)  
    train_data['h2'] = train_data.apply(lambda row: h2.get(row.doc_id,0), axis = 1)  
    train_data['h3'] = train_data.apply(lambda row: h3.get(row.doc_id,0), axis = 1)  
    train_data['h4'] = train_data.apply(lambda row: h4.get(row.doc_id,0), axis = 1)  
    train_data['h5'] = train_data.apply(lambda row: h5.get(row.doc_id,0), axis = 1)  
    train_data['h6'] = train_data.apply(lambda row: h6.get(row.doc_id,0), axis = 1)  
                        
                
#     train_data['url'] = train_data.apply(lambda row: urls.get(row.doc_id,''), axis = 1)  
#     train_data['hrefs'] = train_data.apply(lambda row: hrefs.get(row.doc_id,''), axis = 1)  
#     train_data['hrefs_n'] = train_data.apply(lambda row: hrefs_n.get(row.doc_id,''), axis = 1)  
#     train_data['description'] = train_data.apply(lambda row: description.get(row.doc_id,''), axis = 1)  
#     train_data['keywords'] = train_data.apply(lambda row: keywords.get(row.doc_id,''), axis = 1)  


In [ ]:
len(description),len(keywords)

In [ ]:

save_obj(urls, 'urls')
save_obj(hrefs, 'hrefs')
save_obj(hrefs_n, 'hrefs_n')
save_obj(description, 'description')
save_obj(keywords, 'keywords')
save_obj(p, 'p')
save_obj(h1, 'h1')
save_obj(h2, 'h2')
save_obj(h3, 'h3')
save_obj(h4, 'h4')
save_obj(h5, 'h5')
save_obj(h6, 'h6')

In [7]:
urls=load_obj( 'urls')
hrefs=load_obj( 'hrefs')
hrefs_n=load_obj( 'hrefs_n')
description= load_obj( 'description')
keywords = load_obj( 'keywords')
p =load_obj( 'p')
h1 =load_obj( 'h1')
h2 =load_obj( 'h2')
h3 =load_obj( 'h3')
h4 =load_obj( 'h4')
h5 =load_obj( 'h5')
h6 =load_obj( 'h6')

In [8]:
#если загружать
train_data['p'] = train_data.apply(lambda row: p.get(row.doc_id,0), axis = 1)  
train_data['h1'] = train_data.apply(lambda row: h1.get(row.doc_id,0), axis = 1)  
train_data['h2'] = train_data.apply(lambda row: h2.get(row.doc_id,0), axis = 1)  
train_data['h3'] = train_data.apply(lambda row: h3.get(row.doc_id,0), axis = 1)  
train_data['h4'] = train_data.apply(lambda row: h4.get(row.doc_id,0), axis = 1)  
train_data['h5'] = train_data.apply(lambda row: h5.get(row.doc_id,0), axis = 1)  
train_data['h6'] = train_data.apply(lambda row: h6.get(row.doc_id,0), axis = 1)  
                        
                
train_data['url'] = train_data.apply(lambda row: urls.get(row.doc_id,''), axis = 1)  
train_data['hrefs'] = train_data.apply(lambda row: hrefs.get(row.doc_id,''), axis = 1)  
train_data['hrefs_n'] = train_data.apply(lambda row: hrefs_n.get(row.doc_id,''), axis = 1)  
train_data['description'] = train_data.apply(lambda row: description.get(row.doc_id,''), axis = 1)  
train_data['keywords'] = train_data.apply(lambda row: keywords.get(row.doc_id,''), axis = 1)  

In [366]:
new_meta = load_obj('tem1')
new_meta2 = load_obj('themes_Dasha')
new_meta3 = load_obj('download2_Anya_')
keys = list(new_meta3.keys())
for key in keys:
    new_meta3[int(key)]=new_meta3[key]
    del new_meta3[key]
new_meta.update(new_meta2)
new_meta.update(new_meta3)

In [367]:
for key in new_meta.keys():
    s = new_meta[key]
    s = ' '.join(s)
    s = s[:s.find('Совпад')]
    s=s.replace('\n'," ")                                                    
    s=s.replace('\t'," ")  
    s=s.replace('.'," ")  
    s=s.replace('%'," ")  
    
    s = ''.join(i for i in s if not i.isdigit())
    s=s.replace('Тематика:'," ")  
    s=s.replace('Совпадение:'," ")  
    s = s.strip()
    s=s.replace(' ',"а")  
    s=s.replace('/'," ")  
    new_meta[key] = s

In [419]:
new_meta3[10001]

['\n\n                        Тематика:\n                                                    Top/Computers/Programming\n\n\n                        Совпадение:\n                        60.1%\n\t\t\t\t\t\n',
 '\n\n                        Тематика:\n                                                    Top/Business/News_and_Media\n\n\n                        Совпадение:\n                        54.3%\n\t\t\t\t\t\n',
 '\n\n                        Тематика:\n                                                    Top/Computers/Hacking\n\n\n                        Совпадение:\n                        53.7%\n\t\t\t\t\t\n']

In [368]:
train_data['new_meta'] = train_data.apply(lambda row: new_meta.get(row.doc_id,''), axis = 1)  

In [ ]:
save_obj(train_data, name + 'lem_meta_hrefs')

In [9]:
doc_to_title = {}
with open('docs_titles.tsv', encoding = 'UTF-8') as f:
    for num_line, line in enumerate(f):
        if num_line == 0:
            continue
        data = line.strip().split('\t', 1)
        doc_id = int(data[0])
        if len(data) == 1:
            title = ''
        else:
            title = data[1]
        doc_to_title[doc_id] = title
if submit:
    test_doc_titles = []
    for id in test_data.doc_id.values:
        test_doc_titles.append(doc_to_title[id])
    test_data['title'] = test_doc_titles
    print(len(test_doc_titles))
else:
    doc_titles = []
    for id in train_data.doc_id.values:
        doc_titles.append(doc_to_title[id])
    train_data['title'] = doc_titles
    print(len(doc_titles))


11690


In [10]:
if submit:
    test_data['lem_title'] = test_data.apply(lambda row:lemmatize(row.title) , axis = 1)  
else:
    train_data['lem_title'] = train_data.apply(lambda row:lemmatize(row.title) , axis = 1)  

In [11]:
if submit:
    test_data['sum_title'] = test_data.apply(lambda row: row.lem_title +row.description + row.keywords , axis = 1)  
else:
    train_data['sum_title'] = train_data.apply(lambda row: row.lem_title +row.description + row.keywords , axis = 1)  


In [ ]:

# from tqdm.notebook import tqdm
# tqdm.pandas()
# train_data['text'] = train_data.progress_apply(lambda row: re.sub(r'\d+', '', row.text) , axis = 1)  

In [12]:
def get_splits(data, n=3, random_state=None):
    groups = train_data.group_id.unique()
    kf = KFold(n_splits=n, shuffle=True, random_state=random_state)
    for i,j in kf.split(groups):
        train, test = data.group_id.isin(groups[i].reshape(-1)), data.group_id.isin(groups[j].reshape(-1))
        yield (train[train].index.values, test[test].index.values)

In [13]:
#vectorizer = CountVectorizer(ngram_range = (1,3))#max_features = 70)
stop_words =['без','в','во','для','до','за','И','из','к','ко','меж','на','над','о','об','обо','от','ото','по','под','при','про','с', 'а','же','и','ибо','или','как','но','раз','чем','что','не','ни','ли','ль','вот','вон','то','ну','как','же','ни','уж','ну','я', 'ты', 'он', 'мы', 'вы', 'её', 'их','кто','тот']

 
#vectorizer = TfidfVectorizer(preprocessor=lemmatize,stop_words=stop_words)#max_features = 70)
vectorizer = CountVectorizer()#max_features = 70)
#vectorizer = TfidfVectorizer(stop_words=stop_words)#max_features = 70)
#vectorizer = TfidfVectorizer()#max_features = 70)

In [154]:
all_groups = np.array(list(range(1,310))).reshape(-1,1)
i = OneHotEncoder()
i.fit(all_groups)
if submit:
    groups_one_hot_test   = i.transform(test_data.group_id.values.reshape(-1,1))
else:
    groups_one_hot_train = i.transform(train_data.group_id.values.reshape(-1,1))
train_data.group_id.values.reshape(-1,1)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


array([[  1],
       [  1],
       [  1],
       ...,
       [129],
       [129],
       [129]], dtype=int16)

In [281]:
train_data.new_meta.iloc[111]

'Top Computers Programming                                                                                                                                               Top Games Video_Games                                                                                                                                               Top Computers Emulators                                                              '

In [ ]:
def get_top_n_words(corpus, n=None):
    """
    List the top n words in a vocabulary according to occurrence in a text corpus.

    get_top_n_words(["I love Python", "Python is a language programming", "Hello world", "I love the world"]) -> 
    [('python', 2),
     ('world', 2),
     ('love', 2),
     ('hello', 1),
     ('is', 1),
     ('programming', 1),
     ('the', 1),
     ('language', 1)]
    """
    vec = CountVectorizer(stop_words=stop_words).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in     vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)

    return [i[0] for i in words_freq[:n]]

In [ ]:
flatten = lambda l: [item for sublist in l for item in sublist[1:3]]
train_data['hrefs'] = train_data.apply(lambda row: flatten(row.hrefs), axis = 1)  

In [ ]:
train_data['url'] = train_data.apply(lambda row: ' '.join(row.url), axis = 1)  

In [383]:
vectorizer2 = CountVectorizer()
vectorizer2.fit(train_data.new_meta)
#vectorizer2 = CountVectorizer()


CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [420]:
def get_features(data):
    scaler = StandardScaler(with_mean=False)
    number = 1
    data['first_pop'] =0
    for group in tqdm(data.group_id.unique()):
        number +=1
        elements_in_same_group = data.group_id == group
        #words = data.title[elements_in_same_group] + data.text[elements_in_same_group]
        words = data.lem_title[elements_in_same_group] #data.lem_title[elements_in_same_group]
        vectorizer.fit(words)
        texts_vectorized_titles = vectorizer.transform(data.lem_title[elements_in_same_group])
        texts_vectorized_titles = scaler.fit_transform(texts_vectorized_titles)
        texts_vectorized_texts = vectorizer.transform(data.text[elements_in_same_group])
        texts_vectorized_texts = scaler.fit_transform(texts_vectorized_texts)
        texts_vectorized_keywords = vectorizer.transform(data.keywords[elements_in_same_group])
        texts_vectorized_keywords = scaler.fit_transform(texts_vectorized_keywords)
        texts_vectorized_description= vectorizer.transform(data.description[elements_in_same_group])
        texts_vectorized_description= scaler.fit_transform(texts_vectorized_description)
    
        texts_vectorized_sumtitles = vectorizer.transform(data.sum_title[elements_in_same_group])
        texts_vectorized_sumtitles = scaler.fit_transform(texts_vectorized_sumtitles)
        
# #        vectorizer2.fit(data.new_meta[elements_in_same_group])
#        texts_vectorized_new_meta= vectorizer2.transform(data.new_meta[elements_in_same_group])
#        texts_vectorized_new_meta = csr_matrix(np.abs(texts_vectorized_new_meta.toarray()-1))
#        texts_vectorized_new_meta= scaler.fit_transform(texts_vectorized_new_meta)
#         urls = data.url[elements_in_same_group] + data.hrefs[elements_in_same_group]
#         urls = data.hrefs[elements_in_same_group]
# #        print(urls)
#         vectorizer2 = TfidfVectorizer()#max_features = 70)
#         vectorizer2.fit(urls)
#         texts_vectorized_urls = vectorizer2.transform(urls)
#         texts_vectorized_urls = scaler.fit_transform(texts_vectorized_urls)
        
        dist_matr = pairwise_distances(texts_vectorized_titles, texts_vectorized_titles, metric = 'cosine')
        dist_matr2 = pairwise_distances(texts_vectorized_texts, texts_vectorized_texts, metric = 'cosine')
        dist_matr3 = pairwise_distances(texts_vectorized_texts, texts_vectorized_titles, metric = 'cosine')
        dist_matr4 = pairwise_distances(texts_vectorized_keywords, texts_vectorized_keywords, metric = 'cosine')
        dist_matr5 = pairwise_distances(texts_vectorized_description, texts_vectorized_description, metric = 'cosine')
        dist_matr6 = pairwise_distances(texts_vectorized_sumtitles, texts_vectorized_texts, metric = 'cosine')
        dist_matr7 = pairwise_distances(texts_vectorized_keywords, texts_vectorized_titles, metric = 'cosine')
        dist_matr8 = pairwise_distances(texts_vectorized_description, texts_vectorized_titles, metric = 'cosine')
        #clustering = DBSCAN(eps = 1.3, min_samples=5).fit(dist_matr)
        #clf = IsolationForest(random_state=0).fit_predict(dist_matr)

        dist_matr_sorted = np.sort(dist_matr,axis=1)[:,0:21]
        dist_matr_sorted2 = np.sort(dist_matr2,axis=1)[:,0:21]
        dist_matr_sorted3 = np.sort(dist_matr3,axis=1)[:,0:21]
        dist_matr_sorted4 = np.sort(dist_matr4,axis=1)[:,:-1][:,0:21]
        dist_matr_sorted5 = np.sort(dist_matr5,axis=1)[:,:-1][:,0:21]
        dist_matr_sorted6 = np.sort(dist_matr6,axis=1)[:,:-1][:,0:21]
        dist_matr_sorted7 = np.sort(dist_matr7,axis=1)[:,:-1][:,0:21]
        dist_matr_sorted8 = np.sort(dist_matr8,axis=1)[:,:-1][:,0:21]
#         dist_matr_sorted = np.sort(dist_matr,axis=1)[:,0:min(50,dist_matr.shape[1])]
#         dist_matr_sorted2 = np.sort(dist_matr2,axis=1)[:,0:min(50,dist_matr2.shape[1])]
#         dist_matr_sorted3 = np.sort(dist_matr3,axis=1)[:,0:min(50,dist_matr3.shape[1])]

#         if dist_matr_sorted.shape[1] < 50:
#             dist_matr_sorted = np.hstack((dist_matr_sorted, np.tile(dist_matr_sorted[:,-1:], (1, 50-dist_matr_sorted.shape[1]))))
#             dist_matr_sorted2 = np.hstack((dist_matr_sorted2, np.tile(dist_matr_sorted2[:,-1:], (1, 50-dist_matr_sorted2.shape[1]))))
#             dist_matr_sorted3 = np.hstack((dist_matr_sorted3, np.tile(dist_matr_sorted3[:,-1:], (1, 50-dist_matr_sorted3.shape[1]))))


        mean_dist = dist_matr.mean(axis = 1)
        mean_dist2 = dist_matr2.mean(axis = 1)
        mean_dist3 = dist_matr3.mean(axis = 1)
        mean_dist4 = dist_matr4.mean(axis = 1)
        mean_dist5 = dist_matr5.mean(axis = 1)
        mean_dist6 = dist_matr6.mean(axis = 1)
        mean_dist7 = dist_matr7.mean(axis = 1)
        mean_dist8 = dist_matr8.mean(axis = 1)
        dist_matr_sorted = np.hstack((dist_matr_sorted, mean_dist.reshape(-1,1)))
        dist_matr_sorted = np.hstack((dist_matr_sorted, mean_dist2.reshape(-1,1)))
        dist_matr_sorted = np.hstack((dist_matr_sorted, mean_dist3.reshape(-1,1)))
        dist_matr_sorted = np.hstack((dist_matr_sorted, mean_dist4.reshape(-1,1)))
        dist_matr_sorted = np.hstack((dist_matr_sorted, mean_dist5.reshape(-1,1)))
        dist_matr_sorted = np.hstack((dist_matr_sorted, mean_dist6.reshape(-1,1)))
        dist_matr_sorted = np.hstack((dist_matr_sorted, mean_dist7.reshape(-1,1)))
        dist_matr_sorted = np.hstack((dist_matr_sorted, mean_dist8.reshape(-1,1)))
        
#         pop_words = get_top_n_words(words,n=100)
#         for pop_word in pop_words:
#             data.loc[elements_in_same_group,'first_pop']= data[elements_in_same_group].apply(lambda row: int(pop_word in row.text) , axis = 1)  
#             pop  = csr_matrix(data[elements_in_same_group].first_pop.values)
#             print(pop_word,pop.toarray())
#             dist_matr_sorted = np.hstack((dist_matr_sorted, pop.toarray().reshape(-1,1)))
        dist_matr_sorted = np.hstack((dist_matr_sorted, dist_matr_sorted2))
        dist_matr_sorted = np.hstack((dist_matr_sorted, dist_matr_sorted3))
        dist_matr_sorted = np.hstack((dist_matr_sorted, dist_matr_sorted4))
        dist_matr_sorted = np.hstack((dist_matr_sorted, dist_matr_sorted5))
        dist_matr_sorted = np.hstack((dist_matr_sorted, dist_matr_sorted6))
        dist_matr_sorted = np.hstack((dist_matr_sorted, dist_matr_sorted7))
        dist_matr_sorted = np.hstack((dist_matr_sorted, dist_matr_sorted8))
        n =  data.hrefs_n[elements_in_same_group].values
        dist_matr_sorted = np.hstack((dist_matr_sorted, n.reshape(-1,1)))
        n =  data.p[elements_in_same_group].values
        dist_matr_sorted = np.hstack((dist_matr_sorted, n.reshape(-1,1)))
        n =  data.h1[elements_in_same_group].values
        dist_matr_sorted = np.hstack((dist_matr_sorted, n.reshape(-1,1)))
        n =  data.h2[elements_in_same_group].values
        dist_matr_sorted = np.hstack((dist_matr_sorted, n.reshape(-1,1)))
        n =  data.h3[elements_in_same_group].values
        dist_matr_sorted = np.hstack((dist_matr_sorted, n.reshape(-1,1)))
        n =  data.h4[elements_in_same_group].values
        dist_matr_sorted = np.hstack((dist_matr_sorted, n.reshape(-1,1)))
        n =  data.h5[elements_in_same_group].values
        dist_matr_sorted = np.hstack((dist_matr_sorted, n.reshape(-1,1)))
        n =  data.h6[elements_in_same_group].values
        dist_matr_sorted = np.hstack((dist_matr_sorted, n.reshape(-1,1)))
# #        dist_matr_sorted = np.hstack((dist_matr_sorted, clustering.labels_.reshape(-1,1)))
      #  dist_matr_sorted = np.hstack((dist_matr_sorted, texts_vectorized_new_meta.toarray()))

        if number == 2 :
            features_by_dist =  dist_matr_sorted
        else:
            features_by_dist = vstack((features_by_dist, dist_matr_sorted))
    return features_by_dist.tocsr()
            
#         print(vectorizer.get_feature_names())
#         print(word_vectors_s2['ваза'])
#         coord = np.array([word_vectors_s2[i] for i in word_vectors_s2.index2entity[start_wn:start_wn+words_n]])
#         fig, ax = plt.subplots()
#         coord = coord.T
#         ax.scatter(coord[0],coord[1])
#         for i in range(words_n):
#             plt.annotate(word_vectors_s2.index2entity[start_wn+i], (coord[0][i],coord[1][i]) )
#         plt.show()
#         break
#         vector_size=5
#         e = []
#         e.append(list(vectorizer.vocabulary_.keys()))
#         w2v = Word2Vec(e, min_count=1, size=vector_size, iter=10)
#         word_vectors_s2 = w2v.wv
#         #print(data.lem_title[elements_in_same_group][0])
#         vectors = []
#         for i in vectorizer.inverse_transform(texts_vectorized_texts):
#             tmp2 = np.zeros(vector_size,dtype=np.float64)
#             for j in i[:3]:
#                 tmp2 += np.array(word_vectors_s2[j])
#                 break
# #             if(len(i)!=0):
# #                 tmp2/len(i)
#             vectors.append(tmp2)
                
#         new = np.array(vectors)
                

In [421]:
if submit:
    features_by_dist = get_features(test_data)
    features_by_dist = hstack([features_by_dist, groups_one_hot_test])
    features_by_dist = features_by_dist.tocsr()
else:
    features_by_dist = get_features(train_data)
    features_by_dist = hstack([features_by_dist, groups_one_hot_train])
    features_by_dist = features_by_dist.tocsr()
    target = train_data.target.values

In [ ]:
from xgboost import XGBClassifier
from scipy import stats
from sklearn.neighbors import KNeighborsClassifier
#VALIDATION
max_score = 0
scores = []
pca = PCA(n_components=4)
#j = pca.fit_transform(features_by_dist.toarray())
j = features_by_dist
thre = False 
for i in tqdm(range(20)):
    
    for train, test in get_splits(train_data,n=3,random_state=i):
        if thre:
            pass
    #        model1 = RandomForestClassifier(n_jobs=6,n_estimators=80)
            #model = GradientBoostingClassifier(n_estimators=20, learning_rate=0.2, max_depth=9)
    #        model2 = XGBClassifier()
    #        model3 = LogisticRegression(n_jobs=6)
    #        model.fit(features_by_dist[train], target[train])
    #        model1.fit(j[train], target[train])
    #        model2.fit(j[train], target[train])
    #        model3.fit(j[train], target[train])
#             pred1 = model.predict(j[test])               
#             pred2 = model.predict(j[test])               
#             pred3 = model.predict(j[test])               
#             pred = (pred1 + pred2+pred3)>1
#             pred = pred.astype(int)
#            pred = model.predict(j[test])               
        else:
            #model = KNeighborsClassifier(n_neighbors=3)
            model = RandomForestClassifier(n_jobs=6,n_estimators=1000)
            model.fit(j[train], target[train])
            pred = model.predict(j[test])               
        score = f1_score(pred,train_data.target[test])
        scores.append(score)
        print(score, end=' ')
    print()

print('\nAvg score:' ,sum(scores)/len(scores))

0.7542955326460482 0.7695150115473441 0.7397397397397396 
0.7294569918308506 0.7547495682210708 0.7420289855072464 
0.7911290322580646 0.7591318157755426 0.6893939393939393 
0.7940074906367041 0.7447698744769875 0.7174472465259907 
0.7183348095659876 0.7554060547813551 0.782051282051282 


In [ ]:
#SAVE model
from sklearn.ensemble import RandomForestClassifier 
# model = RandomForestClassifier(n_jobs=6,n_estimators=1000)
# model.fit(features_by_dist, target)
# save_obj(model, 'model')

model1 = RandomForestClassifier(n_jobs=6,n_estimators=80)
#model = GradientBoostingClassifier(n_estimators=20, learning_rate=0.2, max_depth=9)
model2 = XGBClassifier()
model3 = LogisticRegression(n_jobs=6)
#        model.fit(features_by_dist[train], target[train])
model1.fit(features_by_dist, target)
model2.fit(features_by_dist, target)
model3.fit(features_by_dist, target)
save_obj(model1, 'model1')
save_obj(model2, 'model2')
save_obj(model3, 'model3')

In [ ]:
#SUBMIT
from sklearn.ensemble import RandomForestClassifier 
# print(type(features_by_dist))
# model = load_obj('model') 
# pred = model.predict(features_by_dist)               
# pred.shape
model1 = load_obj('model1') 
model2 = load_obj('model2') 
model3 = load_obj('model3') 
pred1 = model1.predict(features_by_dist)               
pred2 = model2.predict(features_by_dist)               
pred3 = model3.predict(features_by_dist)               
pred = (pred1 + pred2+pred3)>1
pred = pred.astype(int)

In [ ]:
test_data['target'] = pred
test_data.to_csv('sub_3with_meta',index=False,columns=['pair_id','target'])

In [ ]:
from time import sleep
gr = int(input())
print(gr)
pca = PCA(n_components=components=2)
i = pca.fit_transform(features_by_dist[train_data[train_data.group_id==gr].index.values].toarray())
a = plt.subplot()
a.scatter(i[:, 0], i[:, 1], c=target[train_data[train_data.group_id==gr].index.values])

In [ ]:
save_obj(test_data,'test')